## Task 2

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import svm

train = pd.read_csv('train_features.csv')
test = pd.read_csv('test_features.csv')
trainl = pd.read_csv('train_labels.csv')

pd.set_option('display.max_columns',None)

df = test

In [4]:
def getMean(arr,label):
    realMean = (pd.DataFrame(train[label]).mean())[0]
    a = np.array(arr)
    if np.sum(np.isnan(arr)) == a.size:
        return [realMean]
    return [np.nanmean(a)]

def getMax(arr,label):
    realMean = (pd.DataFrame(train[label]).mean())[0]
    a = np.array(arr)
    if np.sum(np.isnan(a)) == a.size:
        return [realMean]
    return [np.nanmax(a)]
    
def getMin(arr,label):
    realMean = (pd.DataFrame(train[label]).mean())[0]
    a = np.array(arr)
    if np.sum(np.isnan(a)) == a.size:
        return [realMean]
    return [np.nanmin(a)]

def getLast(arr,label):
    realMean = (pd.DataFrame(train[label]).mean())[0]
    a = np.array(arr)
    last = realMean
    for x in a:
        if not np.isnan(x):
            last = x
    return [last]

def getTrend(arr,label):
    if (getCount(arr) <= 1):
        return 0.0
    b = np.arange(len(arr))
    a = np.array(arr)
    index = []
    for i in range(0,len(a)):
        if np.isnan(a[i]):
            index.append(i)
    a = np.delete(a,index)
    b = np.delete(b,index)
    if len(a) == 0:
        return [0]
    slope, intercept, r_value, p_value, std_err = stats.linregress(b,a)
    return [slope]

def getCount(arr,label):
    a = np.array(arr)
    return [np.count_nonzero(~np.isnan(a))]


In [5]:
LABELS = ['pid','Age','Temp','RRate','ABPm','ABPd','ABPs','Heartrate','SpO2','EtCO2','PTT','BUN','Lactate',
          'Hgb','HCO3','BaseExcess','Fibrinogen','Phosphate','Phosphate','WBC','Creatinine','PaCO2',
        'AST','FiO2','Platelets','SaO2','Glucose','Magnesium','Potassium','Calcium','Alkalinephos',
          'Bilirubin_direct','Chloride','Hct', 'Bilirubin_total','TroponinI','pH']
VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

In [54]:
# Run it to create features
features = pd.DataFrame()

for x in pd.unique(df['pid']):
    temp = df.loc[df['pid'] == x]
    row = np.array([0])
    for label in LABELS :
        if label == 'pid':
            row[0] = list(temp[label])[0]
        elif label == 'Age':
            row = np.concatenate([row,[list(temp[label])[0]]])
        elif label in VITALS:
            data = temp[label]
            row = np.concatenate([row,getMean(list(data),label)])
            row = np.concatenate([row,getLast(list(data),label)])
            row = np.concatenate([row,getCount(list(data),label)])
            row = np.concatenate([row,getTrend(list(data),label)])
            row = np.concatenate([row,getMin(list(data),label)])
            row = np.concatenate([row,getMax(list(data),label)])
        else:
            data = temp[label]
            row = np.concatenate([row,getCount(list(data),label)])
            row = np.concatenate([row,getMean(list(data),label)])
            row = np.concatenate([row,getLast(list(data),label)])

    row_df = pd.DataFrame([row])
    features = pd.concat([features,row_df],ignore_index=True)


In [55]:
# Standartize
meanArrTrain = np.zeros(len(features.columns))
stdArrTrain = np.zeros(len(features.columns))

for x in range(1,len(features.columns)):
    mean = features.iloc[:,x].mean()
    std = features.iloc[:,x].std()
    meanArrTrain[x-1] = mean
    stdArrTrain[x-1] = std
    features.iloc[:,x] = (features.iloc[:,x] - mean)/std


In [56]:
# Save it to csv file
features.to_csv(r'/home/nicolas/eth/hardBaselineFeaturesTest.csv', index = False, header=True)

# Abgabe

In [5]:
trainX = pd.read_csv('hardBaselineFeaturesTrain.csv')
trainY = pd.read_csv('train_labels.csv')

testX = pd.read_csv('hardBaselineFeaturesTest.csv')

testData = pd.read_csv('test_features.csv')

In [6]:
prediction = pd.DataFrame(columns = ['pid','LABEL_BaseExcess','LABEL_Fibrinogen','LABEL_AST','LABEL_Alkalinephos',
                                      'LABEL_Bilirubin_total','LABEL_Lactate','LABEL_TroponinI','LABEL_SaO2',
                                     'LABEL_Bilirubin_direct','LABEL_EtCO2','LABEL_Sepsis','LABEL_RRate',
                                      'LABEL_ABPm','LABEL_SpO2','LABEL_Heartrate'])

prediction['pid'] = testX['0']

In [14]:
from sklearn.svm import LinearSVC
import math

# LABEL_BaseExcess -> 0
label = 'LABEL_BaseExcess'
y = trainY[label].to_numpy()
svc = svm.SVC(kernel='rbf',C=1, gamma='scale', random_state=0, class_weight='balanced', decision_function_shape='ovr', probability = True)
svc = svc.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = svc.predict_proba((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred[:,1]
print('LABEL_BaseExcess')

# LABEL_Fibrinogen -> 1
label = 'LABEL_Fibrinogen'
y = trainY[label].to_numpy()
svc = svm.SVC(kernel='rbf',C=1, gamma='scale', random_state=0, class_weight='balanced', decision_function_shape='ovr', probability = True)
svc = svc.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = svc.predict_proba((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred[:,1]
print('LABEL_Fibrinogen')


# LABEL_Lactate ->5
label = 'LABEL_Lactate'
y = trainY[label].to_numpy()
svc = svm.SVC(kernel='rbf',C=1, gamma='scale', random_state=0, class_weight='balanced', decision_function_shape='ovr', probability = True)
svc = svc.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = svc.predict_proba((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred[:,1]
print('LABEL_Lactate')

# LABEL_TroponinI ->6
label = 'LABEL_TroponinI'
y = trainY[label].to_numpy()
svc = svm.SVC(kernel='rbf',C=1, gamma='scale', random_state=0, class_weight='balanced', decision_function_shape='ovr', probability = True)
svc = svc.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = svc.predict_proba((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred[:,1]
print('LABEL_TroponinI')

# LABEL_Bilirubin_direct ->8
label = 'LABEL_Bilirubin_direct'
y = trainY[label].to_numpy()
svc = svm.SVC(kernel='rbf',C=1, gamma='scale', random_state=0, class_weight='balanced', decision_function_shape='ovr', probability = True)
svc = svc.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = svc.predict_proba((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred[:,1]
print('LABEL_Bilirubin_direct')

# LABEL_EtCO2 ->9
label = 'LABEL_EtCO2'
y = trainY[label].to_numpy()
svc = svm.SVC(kernel='rbf',C=1, gamma='scale', random_state=0, class_weight='balanced', decision_function_shape='ovr', probability = True)
svc = svc.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = svc.predict_proba((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred[:,1]
print('LABEL_EtCO2')


/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LABEL_AST
LABEL_BaseExcess
LABEL_Fibrinogen


/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LABEL_Alkalinephos


/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LABEL_Bilirubin_total
LABEL_Lactate
LABEL_TroponinI


/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LABEL_SaO2
LABEL_Bilirubin_direct
LABEL_EtCO2


In [15]:
y = trainY['LABEL_Sepsis'].to_numpy()
clf = LinearSVC(random_state=0, tol=1e-5,max_iter=20000, class_weight='balanced')
clf.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = clf.decision_function((testX.loc[:,testX.columns != '0']).to_numpy())
prediction['LABEL_Sepsis'] = ypred

/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [16]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import RidgeCV

RRateLabelMean = trainY['LABEL_RRate'].mean()
RRateLabelvar = trainY['LABEL_RRate'].std()
ABPmLabelMean = trainY['LABEL_ABPm'].mean()
ABPmLabelvar = trainY['LABEL_ABPm'].std()
SpO2LabelMean = trainY['LABEL_SpO2'].mean()
SpO2Labelvar = trainY['LABEL_SpO2'].std()
HeartrateLabelMean = trainY['LABEL_Heartrate'].mean()
HeartrateLabelvar = trainY['LABEL_Heartrate'].std()

Y_RRate = ((trainY['LABEL_RRate']-RRateLabelMean)/RRateLabelvar).to_numpy()
Y_ABPm = ((trainY['LABEL_ABPm']-ABPmLabelMean)/ABPmLabelvar).to_numpy()
Y_SpO2 = ((trainY['LABEL_SpO2']-SpO2LabelMean)/SpO2Labelvar).to_numpy()
Y_Heartrate = ((trainY['LABEL_Heartrate']-HeartrateLabelMean)/HeartrateLabelvar).to_numpy()

# Do RRate prediction
rf = RidgeCV(alphas=[0.001,0.01,0.1,1,10,100,100]).fit(trainX.loc[:,trainX.columns != '0'],Y_RRate)
RRate = rf.predict(testX.loc[:,testX.columns != '0'])*RRateLabelvar + RRateLabelMean

# Do ABPm prediction
rf = RidgeCV(alphas=[0.001,0.01,0.1,1,10,100,100]).fit(trainX.loc[:,trainX.columns != '0'],Y_ABPm)
ABPm = rf.predict(testX.loc[:,testX.columns != '0'])*ABPmLabelvar + ABPmLabelMean

# Do SpO2 prediction
rf = RidgeCV(alphas=[0.001,0.01,0.1,1,10,100,100]).fit(trainX.loc[:,trainX.columns != '0'],Y_SpO2)
SpO2 = rf.predict(testX.loc[:,testX.columns != '0'])*SpO2Labelvar + SpO2LabelMean

# Do Heartrate prediction
rf = RidgeCV(alphas=[0.001,0.01,0.1,1,10,100,100]).fit(trainX.loc[:,trainX.columns != '0'],Y_Heartrate)
Heartrate = rf.predict(testX.loc[:,testX.columns != '0'])*HeartrateLabelvar + HeartrateLabelMean

prediction['LABEL_RRate'] = RRate
prediction['LABEL_ABPm'] = ABPm
prediction['LABEL_SpO2'] = SpO2
prediction['LABEL_Heartrate'] = Heartrate

In [20]:
# LABEL_AST ->2
label = 'LABEL_AST'
y = trainY[label].to_numpy()
clf = LinearSVC(random_state=0, tol=1e-5, max_iter=20000, class_weight='balanced')
clf.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = 1.0-clf.decision_function((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred
print('LABEL_AST')

# LABEL_Alkalinephos ->3
label = 'LABEL_Alkalinephos'
y = trainY[label].to_numpy()
clf = LinearSVC(random_state=0, tol=1e-5, max_iter=20000, class_weight='balanced')
clf.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = 1.0-clf.decision_function((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred
print('LABEL_Alkalinephos')

# LABEL_Bilirubin_total ->4
label = 'LABEL_Bilirubin_total'
y = trainY[label].to_numpy()
clf = LinearSVC(random_state=0, tol=1e-5, max_iter=20000, class_weight='balanced')
clf.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = 1.0-clf.decision_function((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred
print('LABEL_Bilirubin_total')

# LABEL_SaO2 ->7
label = 'LABEL_SaO2'
y = trainY[label].to_numpy()
clf = LinearSVC(random_state=0, tol=1e-5, max_iter=20000, class_weight='balanced')
clf.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = 1.0-clf.decision_function((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred
print('LABEL_SaO2')

/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LABEL_AST


/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LABEL_Alkalinephos


/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LABEL_Bilirubin_total
LABEL_SaO2


/home/nicolas/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [21]:
prediction['LABEL_AST'] = prediction['LABEL_AST'].apply(lambda x: (1/(1+math.exp(-x))))
prediction['LABEL_Alkalinephos'] = prediction['LABEL_Alkalinephos'].apply(lambda x: (1/(1+math.exp(-x))))
prediction['LABEL_Bilirubin_total'] = prediction['LABEL_Bilirubin_total'].apply(lambda x: (1/(1+math.exp(-x))))
prediction['LABEL_SaO2'] = prediction['LABEL_SaO2'].apply(lambda x: (1/(1+math.exp(-x))))
prediction['LABEL_Sepsis'] = prediction['LABEL_Sepsis'].apply(lambda x: (1/(1+math.exp(-x))))

In [22]:
prediction

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0.0,0.320239,0.152314,0.095140,0.128751,0.102675,0.277088,0.041126,0.594671,0.030078,0.030882,0.705408,14.059606,81.725206,99.021878,86.990562
1,10001.0,0.062505,0.048604,0.812803,0.825497,0.826764,0.086166,0.070728,0.847334,0.050008,0.020779,0.580367,17.783333,91.697055,94.498447,96.045491
2,10003.0,0.029350,0.033498,0.867929,0.878496,0.879189,0.271052,0.033423,0.769586,0.025276,0.008347,0.584928,18.031584,79.988071,98.033934,88.505349
3,10004.0,0.030990,0.033522,0.807042,0.818075,0.823872,0.063509,0.044701,0.852623,0.025292,0.042697,0.576833,16.579210,75.987539,95.567204,90.838142
4,10005.0,0.051112,0.007636,0.875342,0.887086,0.883453,0.046173,0.010684,0.863123,0.013374,0.001735,0.586519,19.517245,74.648489,95.814879,61.182972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12659,9989.0,0.669838,0.110411,0.860180,0.872044,0.873399,0.567278,0.004499,0.807638,0.014220,0.002497,0.618619,20.423520,77.614030,95.783606,101.555650
12660,9991.0,0.512694,0.052748,0.350958,0.313333,0.289684,0.168808,0.015027,0.586416,0.048273,0.001411,0.682929,18.461531,95.795017,99.264120,74.393651
12661,9992.0,0.740733,0.052485,0.771623,0.775235,0.771812,0.124242,0.014120,0.628392,0.024073,0.008994,0.625961,18.675887,67.884169,97.478315,83.795500
12662,9994.0,0.434599,0.131934,0.233019,0.191616,0.164800,0.441195,0.058226,0.170431,0.036236,0.041348,0.692075,15.426614,85.563953,98.180500,96.773348


In [ ]:
prediction.to_csv('predictionHardbaseLine.zip', index=False, float_format='%.3f', compression='zip')

In [38]:
# LABEL_AST ->2
label = 'LABEL_AST'
y = trainY[label].to_numpy()
clf = LinearSVC(random_state=0, tol=0.0001, max_iter=2000000, class_weight='balanced')
clf.fit(trainX.loc[:,trainX.columns != '0'].to_numpy(), y)
ypred = clf.decision_function((testX.loc[:,testX.columns != '0']).to_numpy())
prediction[label] = ypred
print('LABEL_AST')
prediction['LABEL_AST'] = prediction['LABEL_AST'].apply(lambda x: (1/(1+math.exp(-x))))

LABEL_AST


In [39]:
prediction

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0.0,0.320239,0.152314,0.956625,0.128751,0.102675,0.277088,0.041126,0.594671,0.030078,0.030882,0.705408,14.059606,81.725206,99.021878,86.990562
1,10001.0,0.062505,0.048604,0.387226,0.825497,0.826764,0.086166,0.070728,0.847334,0.050008,0.020779,0.580367,17.783333,91.697055,94.498447,96.045491
2,10003.0,0.029350,0.033498,0.294939,0.878496,0.879189,0.271052,0.033423,0.769586,0.025276,0.008347,0.584928,18.031584,79.988071,98.033934,88.505349
3,10004.0,0.030990,0.033522,0.394446,0.818075,0.823872,0.063509,0.044701,0.852623,0.025292,0.042697,0.576833,16.579210,75.987539,95.567204,90.838142
4,10005.0,0.051112,0.007636,0.280858,0.887086,0.883453,0.046173,0.010684,0.863123,0.013374,0.001735,0.586519,19.517245,74.648489,95.814879,61.182972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12659,9989.0,0.669838,0.110411,0.308787,0.872044,0.873399,0.567278,0.004499,0.807638,0.014220,0.002497,0.618619,20.423520,77.614030,95.783606,101.555650
12660,9991.0,0.512694,0.052748,0.830792,0.313333,0.289684,0.168808,0.015027,0.586416,0.048273,0.001411,0.682929,18.461531,95.795017,99.264120,74.393651
12661,9992.0,0.740733,0.052485,0.446105,0.775235,0.771812,0.124242,0.014120,0.628392,0.024073,0.008994,0.625961,18.675887,67.884169,97.478315,83.795500
12662,9994.0,0.434599,0.131934,0.900524,0.191616,0.164800,0.441195,0.058226,0.170431,0.036236,0.041348,0.692075,15.426614,85.563953,98.180500,96.773348
